# AHN tiles download and preprocessing

This notebook downloads and pre-processes AHN elevation data for a given set of point cloud tiles.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry as sg
from tqdm.notebook import tqdm
tqdm.pandas()
import pathlib
import os
import requests
import shutil
from urllib.request import urlopen
import json
import sys
import csv
import laspy
import matplotlib.pyplot as plt

from upcp.preprocessing import ahn_preprocessing
from upcp.utils import las_utils

In [ ]:
### SETTINGS ###

# Location of point cloud tiles for which AHN data should be generated
pc_folder = '../datasets/pointclouds/run1/'

# AHN output settings
ahn_version = 'ahn3'  # Either ahn3 or ahn4
ahn_resolution = 0.1  # Resolution for the .npz data
ahn_data_folder = '../datasets/ahn/'  # Location where AHN data will be stored
ahn_subtile_folder = f'{ahn_data_folder}{ahn_version}_subtiles/'  # Location to store AHN subtiles

# https://geotiles.nl/ data source
base_url = f'https://geotiles.nl/{str.upper(ahn_version)}_T/'

# Create folders if they don't exist
pathlib.Path(ahn_data_folder).mkdir(parents=True, exist_ok=True)
pathlib.Path(ahn_subtile_folder).mkdir(parents=True, exist_ok=True)

## Get a list of AHN3 tile names and coordinates

In [ ]:
def get_coordinates_features(features):
    def map_fun(feature):
        minx = feature["geometry"]["coordinates"][0][0][0][0]  # multipolygon causing the nesting
        miny = feature["geometry"]["coordinates"][0][0][0][1]
        maxx = feature["geometry"]["coordinates"][0][0][2][0]
        maxy = feature["geometry"]["coordinates"][0][0][2][1]
        bladnr = feature["properties"]["bladnr"]
        return bladnr, int(np.around(minx)), int(np.around(miny)), int(np.around(maxx)), int(np.around(maxy))
    return [map_fun(feature) for feature in features]

url = "https://geodata.nationaalgeoregister.nl/ahn3/wfs?request=GetFeature&typename=ahn3:ahn3_bladindex&outputFormat=json"
with urlopen(url) as response:
    json_string = response.read()
data = json.loads(json_string)
coordinates = get_coordinates_features(data["features"])

with open(f'{ahn_data_folder}ahn3_tegels.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["bladnr", "min_x", "min_y", "max_x", "max_y"])
    for coord in coordinates:
        csv_writer.writerow(coord)

## Download AHN sub-tiles corresponding to required area

### Load AHN data

In [ ]:
def get_ahn_geometry(row):
    x1, y1, x2, y2 = row[['min_x', 'min_y', 'max_x', 'max_y']]
    return sg.box(x1, y1, x2, y2)

In [ ]:
# Load AHN data
ahn_df = pd.read_csv(f'{ahn_data_folder}ahn3_tegels.csv')
ahn_gdf = gpd.GeoDataFrame({'bladnr': ahn_df['bladnr'],
                            'geometry': ahn_df.progress_apply(get_ahn_geometry, axis=1)},
                           geometry='geometry')
ahn_df = None

### Find AHN subtiles that cover the target area based on CycloMedia tiles

In [ ]:
def get_tile_geometry(row):
    x, y = row[['RD_X', 'RD_Y']]
    return sg.box(x, y, x+50, y+50)

def get_tilecode_poly(tilecode):
    ((x1, y2),(x2, y1)) = las_utils.get_bbox_from_tile_code(tilecode)
    return sg.box(x1, y1, x2, y2)

In [ ]:
# Get a list of target point cloud tiles
pc_tiles = list(las_utils.get_tilecodes_from_folder(pc_folder))

# Generate a GeoDataFrame
pc_tiles_gdf = gpd.GeoDataFrame({'tilecode': pc_tiles,
                                 'geometry': [get_tilecode_poly(tc) for tc in pc_tiles]})

# Generate a merged GeoDataFrame for effiency
merged_poly = pc_tiles_gdf.unary_union
if type(merged_poly) == sg.Polygon:
    merged_poly = sg.MultiPolygon([merged_poly])
merged_pc_tiles = gpd.GeoDataFrame({'geometry': [geom for geom in merged_poly.geoms]})

In [ ]:
# Filter ahn data based on merged target shapes
ahn_gdf['used'] = ahn_gdf.apply(lambda row: (merged_pc_tiles.intersects(row.geometry) 
                                             & ~merged_pc_tiles.touches(row.geometry)).any(),
                                axis=1)
ahn_gdf = ahn_gdf[ahn_gdf['used']]

In [ ]:
# Visualize the result
fig, ax = plt.subplots(1)
ahn_gdf.plot(ax=ax, edgecolor="black", linewidth=0.4, alpha=0.25)
merged_pc_tiles.plot(ax=ax)
ax.set_aspect('equal')
plt.show()

In [ ]:
# Divide AHN tile into 25 subtiles of 1x1.25 km each (see https://geotiles.nl/)
Wst = 1000  # Width of subtile
Hst = 1250  # Height of subtile

subtile_gdf = []

for _, row in ahn_gdf.iterrows():
    (x_min, y_min, x_max, y_max) = row.geometry.bounds
    xs = np.arange(x_min, x_max, Wst, dtype=int)
    ys = np.arange(y_max - Hst, y_min-0.01, -Hst, dtype=int)
    tiles = [sg.box(x, y, x+Wst, y+Hst) for y in ys for x in xs]
    subtile_gdf.append(gpd.GeoDataFrame({'bladnr': [row.bladnr]*len(tiles),
                                         'subtile': range(1, len(tiles)+1),
                                         'geometry': tiles}))

subtile_gdf = gpd.GeoDataFrame(pd.concat(subtile_gdf))

In [ ]:
# Keep only subtiles that contain the target point cloud tiles.
subtile_gdf['used'] = subtile_gdf.apply(lambda row: (merged_pc_tiles.intersects(row.geometry) 
                                                     & ~merged_pc_tiles.touches(row.geometry)).any(),
                                        axis=1)
subtile_gdf = subtile_gdf[subtile_gdf['used']]

In [ ]:
# Visualize the result
fig, ax = plt.subplots(1)
subtile_gdf.plot(ax=ax, edgecolor="black", linewidth=0.4, alpha=0.25)
merged_pc_tiles.plot(ax=ax)
ax.set_aspect('equal')
plt.show()

### Download the AHN subtiles

In [ ]:
def download_url(url, root, filename=None):
    """Download a file from a url and place it in root.
    Args:
        url (str): URL to download file from
        root (str): Directory to place downloaded file in
        filename (str, optional): Name to save the file under. If None, use the basename of the URL
    """

    root = os.path.expanduser(root)
    if not filename:
        filename = os.path.basename(url)
    fpath = os.path.join(root, filename)

    os.makedirs(root, exist_ok=True) 
    
    # make an HTTP request within a context manager
    with requests.get(url, stream=True) as r:
        # check header to get content length, in bytes
        total_length = int(r.headers.get("Content-Length"))

        # implement progress bar via tqdm
        with tqdm.wrapattr(r.raw, "read", total=total_length, desc="") as raw:
            # save the output to a file
            with open(fpath, 'wb') as output:
                shutil.copyfileobj(raw, output)

In [ ]:
# Generate file names
subtile_gdf['filename'] = subtile_gdf.progress_apply(
                                    lambda row: f"{str.upper(row['bladnr'])}_{row['subtile']:02d}.LAZ",
                                    axis=1)

# Get list of required files
req_files = set(subtile_gdf['filename'].values)

# First check if any of the files already exist
current_files = set(file.name for file in pathlib.Path(ahn_subtile_folder).glob('*.LAZ'))
req_files = req_files.difference(current_files)

In [ ]:
# Download the remaining files
file_tqdm = tqdm(req_files, unit='file')
for file in file_tqdm:
    url = f'{base_url}{file}'
    file_tqdm.set_postfix_str(url)
    download_url(url, ahn_subtile_folder, file)

## Pre-process the AHN data

In [ ]:
def match_subtile(row):
    target_df = subtile_gdf[subtile_gdf.contains(row.geometry)]
    if len(target_df) == 0:  # Shouldn't happen
        return None
    else:
        return target_df.iloc[0]['filename']

In [ ]:
ahn_laz_folder = f'{ahn_data_folder}{ahn_version}_laz/'
ahn_npz_folder = f'{ahn_data_folder}{ahn_version}_npz/'

In [ ]:
# Match point cloud tiles to AHN subtiles
pc_tiles_gdf['subtile'] = pc_tiles_gdf.progress_apply(match_subtile, axis=1)

In [ ]:
# Generate .laz tiles
pbar = tqdm(total=len(pc_tiles_gdf))
for subtile in pc_tiles_gdf['subtile'].unique():
    ahn_cloud = laspy.read(f'{ahn_subtile_folder}{subtile}')
    ahn_cloud = laspy.convert(ahn_cloud, point_format_id=3, file_version='1.2')
    for pc_tile in pc_tiles_gdf[pc_tiles_gdf['subtile'] == subtile]['tilecode'].values:
        pc_path = f'{pc_folder}dummy_{pc_tile}.laz'
        ahn_preprocessing.clip_ahn_las_tile(ahn_cloud, pc_path, out_folder=ahn_laz_folder)
        pbar.update(1)
pbar.close()

In [ ]:
# Generate .npz data for all tiles
files = list(pathlib.Path(ahn_laz_folder).glob('ahn_*.laz'))
pathlib.Path(ahn_npz_folder).mkdir(parents=True, exist_ok=True)

file_tqdm = tqdm(files, unit='file', smoothing=0)
for file in file_tqdm:
    ahn_preprocessing.process_ahn_las_tile(
                            file, out_folder=ahn_npz_folder,
                            resolution=ahn_resolution)

In [ ]:
# Alternative, uses parallel processing
!python ../../Urban_PointCloud_Processing/scripts/ahn_batch_processor.py --in_folder {ahn_laz_folder} --out_folder {ahn_npz_folder} --resume --workers 4

## Clean-up

In principle, you can now safely delete the AHN .laz files as they are no longer needed. However, the subtiles could be preserve to prevent having to download them again for nearby point cloud tiles.

In [ ]:
print('The following folders can be deleted:')
print(ahn_subtile_folder)
print(ahn_laz_folder)